# Rewrite with realspace_tb
This notebook reproduces the results/data/*.pkl files using realspace_tb.
Files in the repo were obtained using the archive/lattice/ module.

This notebook is meant to run e.g. on google colab from outside the repo. Hence it clones the repo and installs realspace_tb first.

In [ ]:
import os

REPO_URL = "https://github.com/Leonard-P/orbitronics_research_project.git"
REPO_DIR = "/content/orb"

# Fresh clone (idempotent)
if os.path.exists(REPO_DIR):
    !rm -rf $REPO_DIR

!git clone --depth 1 $REPO_URL $REPO_DIR
%cd $REPO_DIR

# Install realspace_tb package
!python -m pip -q install .

In [ ]:
import realspace_tb as tb
import realspace_tb.orbitronics_2d as orb
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import shutil

In [ ]:
def triple_fold_simulation(Lx, Ly, omega, decay_time, data_folder, filename_suffix=""):
    """Perform 3 simulations with increasing time resolution and save data to data_folder."""
    T = 2 * np.pi / omega
    cycles = 17
    T_ramp = 3 * 2 * np.pi / 0.5

    H = orb.LinearFieldHamiltonian(
        orb.HoneycombLatticeGeometry(Lx, Ly),
        orb.RampedACFieldAmplitude(
            E0=1e-3, omega=omega, T_ramp=T_ramp, direction=np.array([0, 1])
        ),
    )

    if not os.path.isdir(data_folder):
        os.makedirs(data_folder)

    for i in range(3):
        # perform 3 simulations, double the time resolution for each
        dt = 1.0 / (omega * 25 * 2**i)

        orbital_polarization = orb.observables.OrbitalPolarizationObservable(
            H.geometry, electron_mass=0.741,
            window=tb.MeasurementWindow(stride=2 ** (i + 1)),
        )

        animation_obs = orb.observables.LatticeFrameObservable(
            H.geometry,
            electron_mass=0.741,
            window=tb.MeasurementWindow(
                start_time=(cycles - 1) * T + T_ramp,
                stride=int(T / dt / 20),
            ),
        )

        rho = H.ground_state_density_matrix(fermi_level=0.0)
        tb.RK4NeumannSolver().evolve(
            rho, H, dt, total_time=cycles * T + T_ramp, tau=decay_time, observables=[orbital_polarization, animation_obs]
        )

        t_vals = orbital_polarization.measurement_times

        # H uses backend. If this is GPU, make sure to convert E_vals to numpy array
        E_vals = H.field_amplitude.at_time(tb.backend.xp().array(t_vals))
        if tb.backend.USE_GPU:
            E_vals = E_vals.get() # convert to numpy

        # This dump is to reproduce the results/data/*.pkl files analyzed in the project report. 
        # THIS IS NOT RECOMMENDED! IT DOES NOT STORE ALL NECESSARY INFORMATION TO OBTAIN THE GEOMETRY.
        # It would be better to save the entire observable objects using pickle.
        
        dump = {
            "Lx": Lx,
            "Ly": Ly,
            "omega": omega,
            "decay_time": decay_time,
            "h": 2.0 / omega,
            "substeps": 50 * 2**i, # The division of dt in substeps is legacy from old lattice2d code
            "E_time": t_vals,
            "E_amplitude": E_vals,
            "orb_pol_values": orbital_polarization.values,
            "animation_values":  {
                "bond_currents": animation_obs.values["currents"],
                "densities": animation_obs.values["densities"],
                "orbital_curl": animation_obs.values["plaquette_oam"],
            }
        }

        fname = f"Lx{Lx}_Ly{Ly}_omega{omega:.3f}_sub{50 * 2**i}{filename_suffix}.pkl"
        path = os.path.join(data_folder, fname)

        with open(path, "wb") as f:
            pickle.dump(dump, f)

In [ ]:
outpath = "../data"

import time
from itertools import product

# Set backend to GPU with single precision as used in the report.
# This requires a compatible GPU (for cupy -> NVIDIA)
tb.backend.set_backend(use_gpu=True, precision="single")

start = time.time()

Lx_range = [11, 19, 29]
Ly_range = [20, 40, 80]
omega_range = [0.5, 1.0, 2.0]

for Lx, Ly, omega in product(Lx_range, Ly_range, omega_range):
    print(f"Lx: {Lx}, Ly: {Ly}, omega: {omega}")
    triple_fold_simulation(Lx, Ly, omega, 14.0, outpath)

print("Parameter grid done, resimulating without decay.")
triple_fold_simulation(19, 40, 1.0, float("inf"), data_folder=outpath, filename_suffix="_nodecay")

print("Done.")
print(f"Simulation took {(time.time() - start) / 60: .2f} minutes.")


shutil.make_archive(outpath, "zip", outpath)


# Autodownload when in Google Colab
try:
    from google.colab import files
    files.download(outpath + ".zip")
except ImportError:
    pass